# ZTF24aahgqwk in NGC 3443

### Observation Notes

Typically a session has 60 30-second exposures in each of r' and g', but starting with
2024-04-21, there are 120 of g', because g' images were getting fainter.

[ZTF24aahgwk Observation Log](https://brianhill.github.io/supernova-observation/analyses/ZTF24aahgqwk/ZTF24aahgqwk_observation_log.html)


In [1]:
# THIS COMMENT IS THE LONGEST A LINE CAN BE AND STILL RENDER COMPLETELY WHEN PRINTING IN LANDSCAPE MODE.

import os

observations_directory = os.path.join(os.path.expanduser('~'), '2024 Sessions')
analysis_directory = os.path.join(os.path.expanduser('~'), 'ZTF24aahgqwk_analysis')

import math
import numpy as np
from astropy import units as u
from astropy.nddata import CCDData
from astropy.io import fits
from ccdproc import ImageFileCollection, combine, subtract_dark, flat_correct # Combiner
import astroalign as aa
import matplotlib.pyplot as plt
%matplotlib inline

# filters

filters = ['g', 'r']
filter_full_names = ["Sloan g'", "Sloan r'"]

SLOAN_G_FILTER = 0
SLOAN_R_FILTER = 1

# exposure durations

light_exposure = 30 * u.second
flat_exposure = 0.1 * u.second
dark_exposure = light_exposure  # our method presumes this equality
bias_exposure = flat_exposure  # our method presumes this equality



## Combine the Calibration Images into Masters

### Calibration Images

The calibration images are in ~/2024 Sessions/2024-04-12/. In turn, ~/2024 Sessions is
actually a soft link to /Volumes/Astronomy Data/2024 Sessions/2024 Sessions.

In [2]:
# the date on which the calibration images were taken

calibration_date = '2024-04-12'

# calibration directory

calibration_directory = os.path.join(observations_directory, calibration_date)

# subdirectory for the 30-second darks

dark_directory = os.path.join(calibration_directory, 'dark')

# subdirectories for the 0.1-second g and r flats

flat_directories_by_filter = {filter:os.path.join(calibration_directory, 'flat', filter)
                              for filter in filters}

# subdirectory for the biases (TheSky Professional Edition may indicate that these are 0.1-second darks)

bias_directory = os.path.join(calibration_directory, 'bias')

# darks

dark_files = ImageFileCollection(dark_directory).files_filtered(include_path='True')
darks = [trimmed_image_reader(file) for file in dark_files]

for dark in darks:
    confirm_fits_header(dark, (1381, 940), 30.0, 3, 0.0, 'dark')

# flats by filter

flat_files_by_filter = {filter:ImageFileCollection(flat_directory).files_filtered(include_path='True')
                        for filter, flat_directory in flat_directories_by_filter.items()}
flats_by_filter = {filter:[trimmed_image_reader(file) for file in flat_files]
                   for filter, flat_files in flat_files_by_filter.items()}

for filter, flats in flats_by_filter.items():
    for flat in flats:
        confirm_fits_header(flat, (1381, 940), 0.1, 3, 0.0, filter)

# biases

bias_files = ImageFileCollection(bias_directory).files_filtered(include_path='True')
biases = [trimmed_image_reader(file) for file in bias_files]

for bias in biases:
    confirm_fits_header(bias, (1381, 940), 0.1, 3, 0.0, 'dark')

# Combine darks, flats, and biases

calibration_combination_method = 'median'  # alternatively, the method can be 'average'

master_dark = combine(darks, method=calibration_combination_method)
master_flats_by_filter = {filter:combine(flats, method=calibration_combination_method)
                         for filter, flats in flats_by_filter.items()}
master_bias = combine(biases, method=calibration_combination_method)

# Perform dark subtraction of the master flats

master_flats_subtracted_by_filter = {filter:subtract_dark(master_flat,
                                                          master_bias,
                                                          data_exposure=flat_exposure,
                                                          dark_exposure=bias_exposure,
                                                          scale=False)
                                     for filter, master_flat in master_flats_by_filter.items()}


Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'.


Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'.


Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'.


INFO:astropy:splitting each image into 2 chunks to limit memory usage to 16000000000.0 bytes.


INFO: splitting each image into 2 chunks to limit memory usage to 16000000000.0 bytes. [ccdproc.combiner]


INFO:astropy:splitting each image into 2 chunks to limit memory usage to 16000000000.0 bytes.


INFO: splitting each image into 2 chunks to limit memory usage to 16000000000.0 bytes. [ccdproc.combiner]


INFO:astropy:splitting each image into 2 chunks to limit memory usage to 16000000000.0 bytes.


INFO: splitting each image into 2 chunks to limit memory usage to 16000000000.0 bytes. [ccdproc.combiner]


## Load and Calibrate Lights; Then Write Them

What follows is a time-consuming loop, done once for each observation date.

Because it is time-consuming, we write the results, so they can just be read back in for alignment.

In [8]:
observation_dates = [
    '2024-03-20',
    '2024-03-21',
    '2024-03-23',
    '2024-03-27',
    '2024-04-02',
    '2024-04-03',
    '2024-04-04',
    '2024-04-06',
    '2024-04-10',
    '2024-04-11',
    '2024-04-13',
    '2024-04-17',
    '2024-04-21',
    '2024-04-22',
    '2024-04-23',
    '2024-04-29',
    '2024-04-30',
    '2024-05-02'
]

# def hot_pixel_reject(image, sigma_multiple):
#     data = image.data
#     width, height = data.shape
#     with np.nditer(data, flags=['multi_index', 'buffered'], op_flags=['readwrite']) as it:
#         for x in it:
#             neighbors = 0.0
#             squares = 0.0
#             count = 0
#             i, j = it.multi_index
#             for ii in [i - 1, i, i + 1]:
#                 # check for out of bounds column index (and skip if necessary)
#                 if ii < 0 or ii >= width:
#                     continue
#                 for jj in [j - 1, j, j + 1]:
#                     # check for out of bounds row index (and skip if necessary)
#                     if jj < 0 or jj >= height:
#                         continue
#                     # check for self (and skip if necessary)
#                     if ii == i and jj == j:
#                         continue
#                     neighbor = data[ii, jj]
#                     neighbors += neighbor
#                     squares += neighbor * neighbor
#                     count += 1
#             neighbors = neighbors / count  # now neighbors is the mean of the neighbors
#             squares = squares / count # now squares is the mean of the squares
#             sigma = math.sqrt((squares - neighbors * neighbors) * count / (count - 1))
#             x[...] = neighbors if abs(x - neighbors) >= sigma_multiple * sigma else x
#     return image


for observation_date in observation_dates:
    
    # subdirectories for the 30-second g and r lights
    
    light_directories_by_filter = {
        filter:light_directory_for_filter(observation_date, filter)
        for filter in filters
    }
    
    # lights by filter
    
    light_files_by_filter = {
        filter:ImageFileCollection(light_directory).files_filtered(include_path='True')
        for filter, light_directory in light_directories_by_filter.items()
    }
    
    lights_by_filter = {
        filter:[trimmed_image_reader(file) for file in light_files]
        for filter, light_files in light_files_by_filter.items()
    }
    
    for filter, lights in lights_by_filter.items():
        for light in lights:
            confirm_fits_header(light, (1381, 940), 30.0, 3, 0.0, filter)
    
    subtracted_lights_by_filter = {
        filter:[subtract_dark(light,
                              master_dark,
                              data_exposure=light_exposure,
                              dark_exposure=dark_exposure,
                              scale=False) for light in lights]
        for filter, lights in lights_by_filter.items()
    }
    
    # Perform flat division
    
    calibrated_lights_by_filter = {
        filter:[flat_correct(light, master_flats_subtracted_by_filter[filter])
                for light in lights]
        for filter, lights in subtracted_lights_by_filter.items()
    }
    
    # The horizontal lines in CMOS output can be removed with median subtraction
    
    # Code would look something like:
    
    # calibrated_medians = [
    #     np.median(sample, axis=1, keepdims=True)
    #     for sample in sample_calibrated_data_list
    # ]
    
    # calibrated_subtracted = [
    #     sample_calibrated_data_list[i] - calibrated_medians[i]
    #     for i in range(len(filters))
    # ]
    
    # Before we align the lights we could try hot pixel rejection
    
    #     hp_rejected_lights_by_filter = {
    #         filter:[hot_pixel_reject(light, 5.0) for light in lights]
    #         for filter, lights in calibrated_lights_by_filter.items()
    #     }
    
    # write the calibrated lights
    
    calibrated_directories_by_filter = {
        filter:calibrated_directory_for_filter(observation_date, filter)
        for filter in filters
    }
    
    for calibrated_directory in calibrated_directories_by_filter.values():
        if not os.path.exists(calibrated_directory):
            os.makedirs(calibrated_directory)
    
    for filter, calibrated_directory in calibrated_directories_by_filter.items():
        lights = lights_by_filter[filter]
        light_files = light_files_by_filter[filter]
        calibrated_lights = calibrated_lights_by_filter[filter]
        for j in range(len(calibrated_lights)):
            # Then we write all the files for that filter
            light_header = lights[j].header
            calibrated_data = calibrated_lights[j]
            calibrated_file = os.path.join(calibrated_directory, os.path.basename(light_files[j]))
            calibrated_file2 = os.path.splitext(calibrated_file)[0] + '_calibrated.fit'
            fits.writeto(calibrated_file2, calibrated_data, light_header, overwrite=True)

Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'.


Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'.


Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'.


Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'.


Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'.


...

Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'.


Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'.


Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'.


Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'.


Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'.


## Re-Load and Align the Lights

What follows is a giant for loop, done once for each observation date.

In [9]:
observation_dates = [
    '2024-03-20',
    '2024-03-21',
    '2024-03-23',
    '2024-03-27',
    '2024-04-02',
    '2024-04-03',
    '2024-04-04',
    '2024-04-06',
    '2024-04-10',
    '2024-04-11',
    '2024-04-13',
    '2024-04-17',
    '2024-04-21',
    '2024-04-22',
    '2024-04-23',
    '2024-04-29',
    '2024-04-30',
    '2024-05-02'
]

# observation_dates = [
#     # '2024-03-20',
#     '2024-03-21',
#     '2024-03-23',
#     # '2024-03-27',
#     # '2024-04-02',
#     # '2024-04-03',
#     # '2024-04-04',
#     # '2024-04-06',
#     # '2024-04-10',
#     # '2024-04-11',
#     '2024-04-13',
#     # '2024-04-17',
#     '2024-04-21',
#     '2024-04-22',
#     '2024-04-23',
#     # '2024-04-29',
#     # '2024-04-30',
#     # '2024-05-02'
# ]

aa.PIXEL_TOL = 2  # consider raising this from the default of 2 due to poor seeing or wind shake

# THIS COMMENT IS THE LONGEST A LINE CAN BE AND STILL RENDER COMPLETELY WHEN PRINTING IN LANDSCAPE MODE.

for observation_date in observation_dates:
    
    # read back in the calibrated lights
    
    calibrated_directories_by_filter = {
        filter:calibrated_directory_for_filter(observation_date, filter)
        for filter in filters
    }

    calibrated_lights_by_filter = {
        filter:[CCDData.read(file, unit=u.adu)
                for file in ImageFileCollection(calibrated_directory).files_filtered(include_path='True')]
        for filter, calibrated_directory in calibrated_directories_by_filter.items()
    }
    
    lights_aligned_with_footprints_by_filter = {
        filter:[
            # somewhat arbitrarily, we will use the image with index 10 as the reference light
            aa.register(light, calibrated_lights_by_filter[filter][10], detection_sigma=3.0)
            for light in lights
        ]
        for filter, lights in  calibrated_lights_by_filter.items()
    }
    
    # write the aligned lights
    
    aligned_directories_by_filter = {
        filter:aligned_directory_for_filter(observation_date, filter)
        for filter in filters
    }

    for filter in filters:
        lights = lights_by_filter[filter]
        light_files = light_files_by_filter[filter]
        lights_aligned_with_footprints = lights_aligned_with_footprints_by_filter[filter]
        aligned_directory = aligned_directories_by_filter[filter]
        for j in range(len(lights_aligned_with_footprints)):
            # Then we write all the files for that filter
            light_header = lights[j][0].header
            light_aligned_data = lights_aligned_with_footprints[j][0]
            aligned_file = os.path.join(aligned_directory, os.path.basename(light_files[j]))
            aligned_file2 = os.path.splitext(aligned_file)[0] + '_aligned.fit'
            fits.writeto(aligned_file2, light_aligned_data, light_header, overwrite=True)

Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'.


Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to -4483194.922 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851220.317 from OBSGEO-[LBH]'.


ValueError: Big-endian buffer not supported on little-endian compiler

## Re-Load and Stack the Lights

What follows is a giant for loop, done once for each observation date.

In [ ]:
for observation_date in []:
    
    # read back in the aligned lights
    
    aligned_directories_by_filter = {
        filter:aligned_directory_for_filter(observation_date, filter)
        for filter in filters
    }
    
    aligned_lights_by_filter = {
        filter:[CCDData.read(file, unit=u.adu)
                for file in ImageFileCollection(aligned_directory).files_filtered(include_path='True')]
        for filter, aligned_directory in aligned_directories_by_filter.items()
    }

    aligned_lights_by_filter = {
        filter:[CCDData.read(file, unit=u.adu)
                for file in ImageFileCollection(aligned_directory).files_filtered(include_path='True')]
        for filter, aligned_directory in aligned_directories_by_filter.items()
    }
    
    stacking_combination_method = 'average'  # alternatively, the method can be 'median'
    
    combined_lights_by_filter = {
        filter:combine(lights, method=stacking_combination_method)
        for filter, lights in aligned_lights_by_filter.items()
    }
    
    # create the directories where the stacked lights will be written

    stacked_directory = os.path.join(analysis_directory, 'stacked')

    if not os.path.exists(stacked_directory):
        os.makedirs(stacked_directory)

    # write the aligned lights

    for filter in filters:
        stacked_header = aligned_lights_by_filter[filter][0].header
        stacked_data = combined_lights_by_filter[filter]
        stacked_file = os.path.join(stacked_directory, observation_date + '-' + filter + '_stacked.fit')
        fits.writeto(stacked_file, stacked_data, stacked_header, overwrite=True)